In [1]:
from dask.distributed import Client

client = Client(n_workers=2, threads_per_worker=2, memory_limit="12GB")
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 4,Total memory: 22.35 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:57861,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 22.35 GiB
Comm: tcp://127.0.0.1:57873,Total threads: 2
Dashboard: http://127.0.0.1:57876/status,Memory: 11.18 GiB
Nanny: tcp://127.0.0.1:57864,


2025-03-27 03:22:24,745 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:57873 (pid=21372) exceeded 95% memory budget. Restarting...
2025-03-27 03:22:24,755 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:57873' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('repartitiontofewer-11e68300b7bb6ee85bc973bd120d1f67', 0)} (stimulus_id='handle-worker-cleanup-1743031344.7554755')
2025-03-27 03:22:24,758 - distributed.scheduler - ERROR - Couldn't gather keys: {('repartitiontofewer-11e68300b7bb6ee85bc973bd120d1f67', 0): 'waiting'}
2025-03-27 03:22:25,932 - distributed.nanny - WARNING - Restarting worker
2025-03-27 03:22:39,343 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:57872 (pid=34568) exceeded 95% memory budget. Restarting...
2025-03-27 03:22:39,353 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:57872' caused the cluster to lose already computed task(s), which will be recomput

In [2]:
ms_file = 'meta_sessions.csv'
ms_attrs_file = 'meta_sessions_attributes.csv'
ms_attrs_definitions = 'meta_sessions_attributes_definitions.csv'

In [3]:
import dask.dataframe as dd
import pandas as pd

In [4]:
ms = dd.read_csv(ms_file)
ms_attrs = dd.read_csv(ms_attrs_file)

In [5]:
with open(ms_attrs_definitions, 'r') as file:
    attrs_indexes = [int(line.split(',')[0].strip('"')) for i, line in enumerate(file) if i > 0]
print(*attrs_indexes, sep=', ')

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28


In [6]:
for idx in attrs_indexes:
    ms_attrs[f'attr_{idx}'] = ms_attrs['value'].where(ms_attrs['session_attribute_definition_id'] == idx, 0)

In [7]:
ms_attrs = ms_attrs.drop(columns=['created_at', 'updated_at', 'session_attribute_definition_id', 'value', 'id'])

In [8]:
full_table = (
    ms_attrs
    .repartition(partition_size='10Mb')
    .merge(
        ms[['id', 'oauth_user_games_id']].repartition(partition_size='10Mb'),
        left_on='session_id',
        right_on='id',
        on='left'
    )
    .drop_duplicates(['session_id'])
)

In [9]:
full_table.to_csv('result_data.csv', single_file=True, index=False)

['C:\\Users\\andrei_rozdestvenski\\bachalor\\result_data.csv']

In [5]:
full_table = dd.read_csv('result_data.csv')

In [7]:
def shift_group(df):
    df = df.sort_values(by='session_id')
    df['metric'] = df['attr_26'].shift(-1)
    return df

In [8]:
full_table = full_table.groupby('oauth_user_games_id').apply(shift_group, meta={**full_table.dtypes.to_dict(), 'metric': 'int64'})

In [9]:
full_table = full_table.persist()

In [17]:
full_table.dropna(subset=['metric']).reset_index(drop=True).to_csv('prepared_data.csv', single_file=True, index=False)

['C:\\Users\\andrei_rozdestvenski\\bachalor\\prepared_data.csv']

In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

In [22]:
full_table = pd.read_csv('prepared_data.csv')

In [23]:
full_table.head()

,session_id,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,...,attr_22,attr_23,attr_24,attr_25,attr_26,attr_27,attr_28,id,oauth_user_games_id,metric
0,33770778,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,33770778,0003de50-43e0-4188-bb76-fa46f7322c23,0.0
1,33774782,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,33774782,0003de50-43e0-4188-bb76-fa46f7322c23,0.0
2,33783006,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,33783006,0003de50-43e0-4188-bb76-fa46f7322c23,0.0
3,504443,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,504443,0005008d-11c6-4e51-97c8-d8acd27c6eb9,0.0
4,505199,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,505199,0005008d-11c6-4e51-97c8-d8acd27c6eb9,0.0


In [24]:
X_df = full_table[[x for x in full_table.columns if x.startswith('attr_')]]
Y_df = full_table[['metric']]

In [26]:
Y_df.loc[:,'metric'] = (Y_df['metric'] > 0).astype(int)

In [27]:
scaler = StandardScaler()
X = scaler.fit_transform(X_df)
Y = Y_df.values

In [36]:
model = Sequential([
    Input((X.shape[1],)),
    Dense(64, activation='tanh'),
    Dense(32, activation='tanh'),
    Dense(1, activation='sigmoid')
])

# AUC - https://www.tensorflow.org/api_docs/python/tf/keras/metrics/AUC
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, stratify=Y)

In [ ]:
history = model.fit(
    X_train, Y_train,
    validation_data=(X_test, Y_test),
    epochs=25
)

Epoch 1/25
 15083/451754 ━━━━━━━━━━━━━━━━━━━━ 6:39 916us/step - accuracy: 0.7632 - auc: 0.6123 - loss: 0.5344